## Parameter Estimation (import / export)
This example describes how to export the parameter estimation setup to a YaML file (or string), so that it can be easily modified later, and then loaded back (or applied to a different model). 

### The Format
When saving the file, it will be exported as a sequence, of experiments of the following form: 

```
name: Experiment
  filename: data.txt
  type: Time-Course
  separator: "\t"
  first_row: 1
  last_row: 102
  header_row: 1
  weight_method: Mean Square
  normalize_per_experiment: true
  mapping:
  - column: '# Time'
    type: time
  - column: Values[F16BP_obs]
    type: dependent
    cn: CN=Root, ...
    object: '[Fru1,6-P2]'
```

the individual fields are: 

* `name`: the name of the experiment
* `type`: `Time-Course` for time course data (requires a mapping of type `time` to be specified), or `Steady-State` for steady state data. 
* `separator`: the separator being used
* `first_row`: the beginning of the experiment in the file (1 based)
* `last_row`: the last row of the experiment 
* `header_row`: (optional) row with header information that can be later used in the `column` field of the mappings. 
* `weight_method`: one of: `Mean`, `Mean Square`, `Standard Deviation` or `Value Scaling`.
* `normalize_per_experiment`: boolean indicating whether experiments should be scaled individual (`True`) or over all defined experimentes (`False`). 
* `mapping`: sequence of column mappings described as follows. 

The mapping descriptions contain the fields: 

* `column`: either an integer index (zero based), describing which column the mapping applies to. If the experiment has header information, the column may be a string with the (case sensitive) header the mapping applies to. 
* `type`: the type of the mapping for this column. One of `ignored`, `time`, `dependent` or `independent`.
* `object`: display name of the element to map to
* `cn`: (optional) the CN to the reference to map to.
  
  For columns of type `dependent` or `independent` at least on of `object` or `cn` needs to be defined. (The `cn` value takes preference). 

* `weight`: may be used for columns of type `dependent` to customize the scale to be applied to the column. If not specified it will be automatically calculated based on the selected `weight_method`. 

### Example
We start by importing basico as usual (should that fail for you just `!pip install copasi-basico`: 

In [1]:
from basico import *

here we load an existing parameter estimation example, included with the distribution as example: 

In [2]:
dm = load_example('PK')

now we can directly export the setup of the experimental data files as yaml string (or if you supply a filename to the function, it will be saved as file): 

In [3]:
yaml_str = save_experiments_to_yaml()
print("\n".join(yaml_str.split('\n')[:24])) # just restricting the amount of yaml to be printed here to the first experiment

- name: Experiment
  filename: e:/development/basico/basico/data\data_2.txt
  type: Time-Course
  separator: "\t"
  first_row: 1
  last_row: 102
  weight_method: Mean Square
  normalize_per_experiment: true
  header_row: 1
  mapping:
  - column: '# Time'
    type: time
  - column: Values[F16BP_obs]
    type: dependent
    cn: CN=Root,Model=Pritchard2002_glycolysis,Vector=Compartments[cytosol],Vector=Metabolites[Fru1\,6-P2],Reference=Concentration
    object: '[Fru1,6-P2]'
  - column: Values[Glu_obs]
    type: dependent
    cn: CN=Root,Model=Pritchard2002_glycolysis,Vector=Compartments[cytosol],Vector=Metabolites[Glc(int)],Reference=Concentration
    object: '[Glc(int)]'
  - column: Values[Pyr_obs]
    type: dependent
    cn: CN=Root,Model=Pritchard2002_glycolysis,Vector=Compartments[cytosol],Vector=Metabolites[pyruvate],Reference=Concentration
    object: '[pyruvate]'


at this point you'd make modifications to you'd want to it. Remember, that the `cn` is optional. The key points to keep in mind: 

* ensure that the row number for the experiment are consistent with the changes you make
* if it is a time course experiment, ensure you have a column of type `time`. 
* the column specifier is either the index of the column, or the name if the experiment has headers. 

once the changes are made, you can load the setup back into the model, using the `load_experiments_from_yaml` function. This will remove all existing experiments from the file first. 

In [4]:
load_experiments_from_yaml(yaml_str)